In [1]:
import folium
import pandas as pd
import gpxpy
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as mpl
import numpy as np
import collections
%matplotlib notebook


In [40]:
gpx = gpxpy.parse(open('activity_4257789098.gpx')) # a gpx file from my garmin watch for a run I did
gpx = gpxpy.parse(open('activity_4334092284.gpx')) # a gpx file from my garmin watch for a run I did
track = gpx.tracks[0]
segment = track.segments[0]

In [41]:
data = []
segment_length = segment.length_3d()

extension_data = collections.defaultdict(list)
for point_idx, point in enumerate(segment.points):
    extensions = point.extensions

    for child in extensions[0].getchildren(): # this gets the extension data (hr and cadence) from the gpx file
        tag = child.tag.rsplit("}", 1)[-1]
        value = child.text
        extension_data[tag].append(float(value))
            

    data.append([point.longitude, point.latitude,
                 point.elevation, point.time, segment.get_speed(point_idx)])
    

columns = ['Longitude', 'Latitude', 'Altitude', 'Time', 'Speed']
df = pd.DataFrame(data, columns=columns)
df['points'] = list(zip(df.Latitude, df.Longitude))

df.Time = pd.to_datetime(df.Time, format = "%Y-%m-%d %H:%M:%S")
df.Time = df.Time.dt.tz_localize(None)
df.Time = (df.Time- df.Time.values[0]).dt.total_seconds()

df['hr'] = extension_data['hr']
df['cad'] = extension_data['cad']
df['hr'] = df.hr.rolling(25, center = True).mean() # smooth the heart rate
df = df.dropna()
df.head()

,Longitude,Latitude,Altitude,Time,Speed,points,hr,cad
12,-122.477009,37.808828,53.200001,27.0,2.757610,"(37.8088282328099, -122.47700943611562)",119.20,80.0
13,-122.477046,37.808899,53.799999,30.0,2.807221,"(37.80889914371073, -122.47704623267055)",121.48,80.0
14,-122.477066,37.808946,53.799999,32.0,2.771378,"(37.80894625000656, -122.47706609778106)",123.88,80.0
15,-122.477077,37.808970,53.799999,33.0,2.747191,"(37.80896980315447, -122.47707657516003)",126.12,80.0
16,-122.477118,37.809062,54.000000,37.0,2.725124,"(37.8090615849942, -122.47711840085685)",128.12,81.0


### Let's re-calculate the speed because I'm not convinced

In [42]:
def running_speed(time, lat, lon, alt):
    """
    Calculates your running speed given a time series of 
    time, lat, lon, and alt coordinates.

    Units of velocity is meters/sec.
    """
    dt = [(ti-tj) for ti, tj in zip(time[1:], time[:-1])]

    X1 = np.array([lat[1:], lon[1:], alt[1:]]).T
    X2 = np.array([lat[:-1], lon[:-1], alt[:-1]]).T
    dx = 1000*haversine(X1, X2) # In units of meters
    
    return np.append([0], dx/dt)

def haversine(X1, X2):
    """
    I know that haversine is overkill...

    Implementation of the haversine foruma to calculate total distance
    at an average altitude. X1 and X2 must be N*3 array of 
    lat, lon, alt. Units of alt is km.
    """
    Re_km = 6371
    R = (Re_km+(X1[:, 2]+X2[:, 2])/2) # Mean distance from Earth's center.
    s = 2*np.arcsin( np.sqrt( np.sin(np.deg2rad(X1[:, 0]-X2[:, 0])/2)**2 + \
                    np.cos(np.deg2rad(X1[:, 0]))*np.cos(np.deg2rad(X2[:, 0]))*\
                    np.sin(np.deg2rad(X1[:, 1]-X2[:, 1])/2)**2 ))
    return R*s

In [43]:
df.Speed = running_speed(df.Time, df.Latitude, df.Longitude, df.Altitude)*2.23694 # convert to mph
df

,Longitude,Latitude,Altitude,Time,Speed,points,hr,cad
12,-122.477009,37.808828,53.200001,27.0,0.000000,"(37.8088282328099, -122.47700943611562)",119.20,80.0
13,-122.477046,37.808899,53.799999,30.0,6.407647,"(37.80889914371073, -122.47704623267055)",121.48,80.0
14,-122.477066,37.808946,53.799999,32.0,6.227279,"(37.80894625000656, -122.47706609778106)",123.88,80.0
15,-122.477077,37.808970,53.799999,33.0,6.262246,"(37.80896980315447, -122.47707657516003)",126.12,80.0
16,-122.477118,37.809062,54.000000,37.0,6.117333,"(37.8090615849942, -122.47711840085685)",128.12,81.0
17,-122.477149,37.809131,54.200001,40.0,6.161257,"(37.80913123860955, -122.47714882716537)",129.88,80.0
18,-122.477209,37.809281,54.799999,46.0,6.590611,"(37.809281442314386, -122.47720934450626)",131.20,80.0
19,-122.477218,37.809305,54.799999,47.0,6.221556,"(37.80930524691939, -122.4772181455046)",132.24,80.0
20,-122.477246,37.809405,55.000000,51.0,6.392319,"(37.80940482392907, -122.477245638147)",133.16,81.0
21,-122.477259,37.809455,55.200001,53.0,6.400965,"(37.809454780071974, -122.47725879773498)",134.04,81.0


In [44]:
bzn = [45.6770, -111.0429]

loc = [df.Latitude.mean(), df.Longitude.mean()]



m = folium.Map(location = loc,zoom_start=14)
def rgb2hex(rgb):
    rgb = [hex(int(256*x)) for x in rgb]
    r, g, b = [str(x)[2:] for x in rgb]
    
    if r == '100':
        r = 'ff'
    if g == '100':
        g = 'ff'
    if b == '100':
        b= 'ff'
#     print(r, g, b)
    return f"#{r.zfill(2)}{g.zfill(2)}{b.zfill(2)}ff"

norm = mpl.colors.Normalize(vmin=0, vmax=max(df.Speed.values)+1)
color_mapper = cm.ScalarMappable(norm = norm,cmap=cm.jet)
rgb_vals = []
for v in df.Speed.values:
    rgb_vals.append(list(color_mapper.to_rgba(v)[0:3]))

colors = [rgb2hex(rgb) for rgb in rgb_vals]

df['Colors'] = colors
df

,Longitude,Latitude,Altitude,Time,Speed,points,hr,cad,Colors
12,-122.477009,37.808828,53.200001,27.0,0.000000,"(37.8088282328099, -122.47700943611562)",119.20,80.0,#000080ff
13,-122.477046,37.808899,53.799999,30.0,6.407647,"(37.80889914371073, -122.47704623267055)",121.48,80.0,#ffea00ff
14,-122.477066,37.808946,53.799999,32.0,6.227279,"(37.80894625000656, -122.47706609778106)",123.88,80.0,#f2fd05ff
15,-122.477077,37.808970,53.799999,33.0,6.262246,"(37.80896980315447, -122.47707657516003)",126.12,80.0,#f5f902ff
16,-122.477118,37.809062,54.000000,37.0,6.117333,"(37.8090615849942, -122.47711840085685)",128.12,81.0,#e8ff0fff
17,-122.477149,37.809131,54.200001,40.0,6.161257,"(37.80913123860955, -122.47714882716537)",129.88,80.0,#ebff0cff
18,-122.477209,37.809281,54.799999,46.0,6.590611,"(37.809281442314386, -122.47720934450626)",131.20,80.0,#ffd700ff
19,-122.477218,37.809305,54.799999,47.0,6.221556,"(37.80930524691939, -122.4772181455046)",132.24,80.0,#f2fd05ff
20,-122.477246,37.809405,55.000000,51.0,6.392319,"(37.80940482392907, -122.477245638147)",133.16,81.0,#ffee00ff
21,-122.477259,37.809455,55.200001,53.0,6.400965,"(37.809454780071974, -122.47725879773498)",134.04,81.0,#ffea00ff


In [45]:
for i, row in df.iterrows():
    folium.CircleMarker(location=row.points,
                        radius=5,
                        popup=str(row.Speed),
                        color=row.Colors, fill = True).add_to(m)
    

m    
